# Module Import

In [ ]:
import os, json, requests, joblib, time

# Preparation

In [ ]:
api_key = 'Your-Api-Key' # Use your api key
base_url = 'https://api.elsevier.com/content/search/sciencedirect'
search_keyword = 'melting point' # Search keyword in this study
Offset = 0
Show = 100
Headers = {'x-els-apikey': api_key, 'Content-Type': 'application/json', 'Accept': 'application/json'}
Journal = 'Journal of Organometallic Chemistry' # Targeted journal in this study, https://www.sciencedirect.com/journal/journal-of-organometallic-chemistry

# Metadata Extraction

In [ ]:
def get_response(url, Data, Headers):
    
    response = requests.put(url, data=json.dumps(Data), headers=Headers)
    response = response.text.replace('false', 'False').replace('true', 'True')
    
    return eval(response)

In [ ]:
def search_journal(year):

    for Offset in range(0, 1000, Show):
        Data = {'qs': search_keyword, 'pub': Journal, 'date': year, 'filter': {"openAccess": True}, 'display': {'show': Show, 'offset': Offset, 'sortBy': 'date'}} 
        save_name = 'metadata/elsevier_journal={}_date={}_offset={}.sav'.format(Journal, year, Offset) # Change to your folder path
        response = get_response(base_url, Data, Headers)

        try:
            print('year =', year, ', offset =', Offset, ', num of articles =', len(response['results']))
            joblib.dump(response['results'], save_name)
            time.sleep(1)
        except KeyError:
            print('nothing more found in this year')
            break

In [ ]:
# Publications published before a year of 2000 were excluded because of extraction accuracy issues. See the article.
for i in range (2000, 2025, 1):
    search_journal(i)

# Article Download

In [ ]:
def download(result):

    doi = result['doi']
    
    file_path = 'metadata/Article/' # Change to your folder path. Create Article folder to download files.
    file_name = '{}{}.xml'.format(file_path, doi.replace('/', '_'))
    print(file_name)
    
    request_url = 'https://api.elsevier.com/content/article/doi/{}?apiKey={}&httpAccept=text%2Fxml'.format(doi, api_key)
    
    with open(file_name, 'w', encoding='utf-8') as f:
        text = requests.get(request_url).text
        f.write(text)

In [ ]:
meta_dir = 'metadata/'
file_list = os.listdir(meta_dir)

for i in file_list:
    try:
        search_results = joblib.load('{}{}'.format(meta_dir, i))
    
        num_of_article = len(search_results)
        print('----- total number of articles:', num_of_article, '-----')
        
        for j in range(0, num_of_article):
            search_result = search_results[j]
            download(search_result)
            print(j+1, '/', num_of_article, ',', search_result['doi'])
        print('done')
    except:
        pass